<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN05/5.3-Convolution_Layer.ipynb)

# Faltungen für Bilder

- Nachdem wir nun verstanden haben, wie Faltungsschichten in der Theorie funktionieren, sind wir nun bereit zu sehen, wie sie in der Praxis funktionieren.
- Aufbauend auf unserer Motivation, Faltungsneuronale Netze als effiziente Architekturen zur Erforschung von Strukturen in Bilddaten, bleiben wir bei Bildern als laufendes Beispiel.


## Die Kreuzkorrelation

- Streng genommen ist die Bezeichnung *Faltungsschichten* falsch, da die Operationen, die sie durchführt genauer als Kreuzkorrelationen definiert ist.
- Basierend auf unserer Beschreibung von Faltungsschichten werden in einer solchen Schicht ein Eingangstensor und ein Kernel-Tensor durch eine (**Kreuzkorrelation**) zu einem Ausgangstensor kombiniert.


Lassen wir die Kanäle erst einmal ausser Acht und sehen wir uns an, wie das 
mit zweidimensionalen Daten und versteckten Repräsentationen funktioniert.

- In der folgenden Abbildung, ist die Eingabe ein zweidimensionaler Tensor mit einer Höhe von 3 und einer Breite von 3.
- Wir markieren die Form des Tensors als $3 \times 3$ oder ($3$, $3$).
- Die Höhe und Breite des Kerns sind beide 2.
- Die Form des *Kernelfensters* (oder *Faltungsfensters*) ist durch die Höhe und Breite des Kerns gegeben (hier ist es $2 \times 2$).

<img src="Bilder/correlation.svg" width="640" height="440" align="center"/>

Zweidimensionale Kreuzkorrelation: Die schattierten Bereiche sind das erste Ausgabeelement sowie die Eingabe- und Kernel-Tensorelemente, die für die Ausgabeberechnung verwendet werden: $0\times0+1\times1+3\times2+4\times3=19$.]


- Bei der zweidimensionalen Kreuzkorrelationsoperation beginnen wir mit dem Faltungsfenster, das sich an der oberen linken Ecke des Eingabetensorsund schieben es über den Eingabetensor, sowohl von links nach rechts als auch von oben nach unten.
- Wenn das Faltungsfenster an eine bestimmte Position gleitet, werden der in diesem Fenster enthaltene Eingabe-Subtensor und der Kernel-Tensor elementweise multipliziert und der resultierende Tensor wird summiert was einen einzigen skalaren Wert ergibt.
- Dieses Ergebnis gibt den Wert des Ausgangstensors an der entsprechenden Stelle. Hier hat der Ausgangstensor eine Höhe von 2 und eine Breite von 2 und die vier Elemente werden abgeleitet aus der zweidimensionalen Kreuzkorrelationsoperation:

$$
0\cdot0+1\cdot1+3\cdot2+4\cdot3=19,\\
1\cdot0+2\cdot1+4\cdot2+5\cdot3=25,\\
3\cdot0+4\cdot1+6\cdot2+7\cdot3=37,\\
4\cdot0+5\cdot1+7\cdot2+8\cdot3=43.
$$



- Beachten Sie, dass entlang jeder Achse die Ausgabegrösse etwas kleiner ist als die Eingabegrösse.
- Da der Kernel eine Breite und Höhe grösser als eins hat, können wir die Kreuzkorrelation nur für Stellen richtig berechnen, an denen der Kernel vollständig in das Bild passt.
- Die Ausgabegrösse ist gegeben durch die Eingabegrösse $n_h \times n_w$ abzüglich der Grösse des Faltungskerns $k_h \times k_w$ über

$$(n_h-k_h+1) \times (n_w-k_w+1).$$

Dies ist der Fall, da wir genügend Platz benötigen, um den Faltungs-Kernel über das Bild zu "verschieben".
Später werden wir sehen, wie man die Grösse durch Auffüllen des Bildes mit Nullen am Rande des Bildes unverändert lassen kann, 
so dass genügend Platz zum Verschieben des Kerns vorhanden ist. 

Als nächstes implementieren wir diesen Prozess in der Funktion `corr2d`, die einen Eingabetensor `X` und einen Kernel-Tensor `K` annimmt akzeptiert und einen Ausgangstensor `Y` zurückgibt.

In [ ]:
import torch

def corr2d(X, K):
    """Compute 2D cross-correlation in PyTorch."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))  # Initialize output tensor

    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i:i + h, j:j + w] * K)  # Element-wise multiplication and sum

    return Y


Wir können den Eingabe-Tensor `X` und den Kernel-Tensor `K` aus der obigen Abbildung konstruieren
um die Rechnung der zweidimensionalen Kreuzkorrelationsoperation zu überprüfen.


In [ ]:
import torch

def corr2d(X, K):
    """Compute 2D cross-correlation in PyTorch."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))  # Initialize output tensor

    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i:i + h, j:j + w] * K)  # Element-wise multiplication and sum

    return Y

# Define input tensors and kernels
X = torch.tensor([[0.0, 1.0, 2.0], 
                  [3.0, 4.0, 5.0], 
                  [6.0, 7.0, 8.0]])

K = torch.tensor([[+1.0, -1.0], 
                  [-1.0, 1.0]])

# Compute cross-correlation
result1 = corr2d(X, K)
print("Result 1:\n", result1)



In [ ]:

# Second input and kernels
X = torch.tensor([[0.0, 1.0, 1.0, 0.0], 
                  [0.0, 1.0, 1.0, 0.0], 
                  [0.0, 1.0, 1.0, 0.0],
                  [0.0, 1.0, 1.0, 0.0]])

Kv = torch.tensor([[+1.0, +1.0], 
                   [-1.0, -1.0]])

Kh = torch.tensor([[+1.0, -1.0], 
                   [+1.0, -1.0]])

K45 = torch.tensor([[+1.0, -1.0, 0.0], 
                    [0.0, +1.0, -1.0],
                    [0.0,  0.0, +1.0]])

# Compute cross-correlation with different kernels
result2 = corr2d(X, Kh)
print("\nResult 2:\n", result2)

result3 = corr2d(X, Kv)
print("\nResult 3:\n", result3)

result4 = corr2d(X, K45)
print("\nResult 4:\n", result4)

In [ ]:
corr2d(X, Kv)

## Faltungsschichten

- Eine Faltungsschicht führt eine Kreuzkorrelation zwischen der Eingabe und dem Kernel und fügt einen skalaren Bias (Offset) hinzu, um eine Ausgabe zu erzeugen.
- Die beiden Parameter einer Faltungsschicht sind der Kernel und die skalare Bias.
- Beim Trainieren von Modellen, die auf Faltungsschichten basieren, werden die Kernel normalerweise zufällig initialisiert,
genau wie bei einer voll verknüpften Schicht.

Wir sind nun bereit, **eine zweidimensionale Faltungsschicht zu implementieren**
basierend auf der oben definierten Funktion `corr2d`.

In der `__init__` Konstruktorfunktion, deklarieren wir `weight` und `bias` als die beiden Modellparameter.
Die Vorwärtsvermehrungsfunktion ruft die Funktion `corr2d` auf und fügt den Bias hinzu.


In [ ]:
import torch
import torch.nn as nn

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        """
        Custom 2D convolution layer (without built-in nn.Conv2d).
        Parameters:
        - kernel_size (tuple): Shape of the convolution kernel (height, width).
        """
        super().__init__()

        # Define the kernel (weight) and bias explicitly
        self.weight = nn.Parameter(torch.randn(kernel_size))  # Learnable weight
        self.bias = nn.Parameter(torch.randn(1))  # Learnable bias

    def forward(self, X):
        """
        Forward pass using manual 2D cross-correlation.
        Parameters:
        - X (torch.Tensor): Input tensor (image).
        Returns:
        - torch.Tensor: Output after convolution.
        """
        return self.corr2d(X, self.weight) + self.bias

    def corr2d(self, X, K):
        """Compute 2D cross-correlation (convolution without flipping)."""
        h, w = K.shape
        Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))  # Output size

        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                Y[i, j] = torch.sum(X[i:i + h, j:j + w] * K)

        return Y


- In $h \times w$-Faltung oder einem $h \times w$-Faltungskern, sind die Höhe und die Breite des Faltungskerns $h$ bzw. $w$.
- Wir bezeichnen auch eine Faltungsschicht mit einem $h \times w$ Faltungskern einfach als $h \times w$-Faltungsschicht.


## Objektkanten-Erkennung in Bildern

Nehmen wir uns einen Moment Zeit, um eine einfache Anwendung einer Faltungsschicht zu analysieren:
**Erkennung des Kante eines Objekts in einem Bild** indem wir den Ort der Pixelveränderung finden.

Zunächst konstruieren wir ein "Bild" aus $6 \times 8$ Pixeln.
Die mittleren vier Spalten sind schwarz (0) und der Rest ist weiss (1).

In [ ]:
# Create a 6x8 tensor filled with ones
X = torch.ones((6, 8))

# Set columns 2 to 5 to zero (equivalent to X[:, 2:6] in TensorFlow)
X[:, 2:6] = 0

# Print the result
print(X)

- Als nächstes konstruieren wir einen Kernel `K` mit einer Höhe von 1 und einer Breite von 2.
- Wenn wir die Kreuzkorrelationsoperation mit der Eingabe durchführen, wenn die horizontal benachbarten Elemente gleich sind,
andernfalls ist die Ausgabe ungleich Null.


In [ ]:
import torch

# Define the horizontal kernel
K_horizontal = torch.tensor([[-1.0, -1.0], 
                             [+1.0, +1.0]])

# Define the vertical kernel
K_vertical = torch.tensor([[+1.0, -1.0], 
                           [+1.0, -1.0]])

# Define the 45-degree diagonal kernel
K_45 = torch.tensor([[+1.0, 0.0], 
                     [0.0, -1.0]])

# Print the tensors
print("K_horizontal:\n", K_horizontal)
print("\nK_vertical:\n", K_vertical)
print("\nK_45:\n", K_45)




Wir sind bereit, die Kreuzkorrelationsoperation durchzuführen
mit den Argumenten "X" (unsere Eingabe) und "K" (unser Kern).
Wie Sie sehen können, **wir erkennen 1 für die Kante von Weiss nach Schwarz
und -1 für die Kante von Schwarz nach Weiss.**

Alle anderen Ausgaben haben den Wert 0.


In [ ]:
Y = corr2d(X, K_vertical)
Y

In [ ]:
Y = corr2d(X, K_horizontal)
Y

- Wir können nun den Kernel auf das transponierte Bild anwenden.
- Wie erwartet, verschwindet er. **Der Kernel "K" erkennt nur vertikale Kanten**.


In [ ]:
corr2d(X.T, K_horizontal)

In [ ]:
X.T

## Kreuzkorrelation und Faltung

Um das Ergebnis der strikten *Faltungsoperation* zu erhalten, müssen wir nur den zweidimensionalen Kernel-Tensor horizontal und vertikal spiegeln und dann die *Kreuzkorrelation* mit dem Eingabetensor durchführen.

- Es ist bemerkenswert, dass beim Deep Learning die Kernel aus den Daten gelernt werden, die Ausgaben von Faltungsschichten unberührt bleiben unabhängig davon, ob diese Schichten entweder die strengen Faltungsoperationen oder die Kreuzkorrelationsoperationen durchführen.

- Im Einklang mit der Standardterminologie der Deep-Learning-Literatur bezeichnen wir die Kreuzkorrelationsoperation weiterhin als als Faltung bezeichnen, auch wenn sie streng genommen etwas anderes ist.

- Ausserdem verwenden wir den Begriff *Element*, um uns auf einen Eintrag (oder eine Komponente) eines beliebigen Tensors, der eine Schichtdarstellung oder einen Faltungskern darstellt.





## Merkmalskarte (feature map) und rezeptives Feld

Die Ausgabe einer Faltungsschicht wird manchmal als **Feature Map** bezeichnet, da sie als
eine gelernte Repräsentation (Merkmal) in den räumlichen Dimensionen (z.B. Breite und Höhe) betrachtet werden kann, welche an die nachfolgende Schicht weitergeben wird.

- Das *rezeptive Feld* (receptive field) eines CNNs besteht aus allen Elementen (aus allen vorhergehenden Schichten), die die Berechnung von $x$ während der Vorwärtspropagation beeinflussen können beeinflussen können.
- Das rezeptive Feld kann grösser sein kann als die tatsächliche Grösse der Eingabe.




Lassen Sie uns weiterhin das obige Besipiel verwenden, um das rezeptive Feld zu erklären.

- Mit dem $2 \times 2$ Faltungs-Kernel, besteht das rezeptive Feld des schattierten Ausgangselements (mit dem Wert $19$) die vier Elemente im schattierten Teil der Eingabe.
- Bezeichnen wir nun die $2 \times 2$ Ausgabe als $\mathbf{Y}$ und betrachten ein tieferes CNN mit einer zusätzlichen $2 \times 2$ Faltungsschicht, die $\mathbf{Y}$ als seine Eingabe nimmt und ein einzelnes Element $z$ ausgibt.
- In diesem Fall das rezeptive Feld von $z$ auf $\mathbf{Y}$ alle vier Elemente von $\mathbf{Y}$ ein, während das rezeptive Feld auf dem Eingang alle neun Eingangselemente umfasst.

Folglich, wenn ein Element in einer Merkmalskarte ein grösseres rezeptives Feld benötigt
um Eingangsmerkmale in einem grösseren Bereich zu erkennen, können wir ein tieferes Netz aufbauen.

## Zusammenfassung

* Die Kernberechnung einer zweidimensionalen Faltungsschicht ist eine zweidimensionale Kreuzkorrelationsoperation. In ihrer einfachsten Form führt sie eine Kreuzkorrelationsoperation an den zweidimensionalen Eingabedaten und dem Kernel durch und fügt dann einen Bias (Offset) hinzu.
* Wir können einen Kernel entwerfen, um Kanten in Bildern zu erkennen.
* Wir können die Parameter des Kernels aus den Daten lernen.
* Mit Kerneln, die aus Daten gelernt wurden, bleiben die Ausgaben von Faltungsschichten unabhängig von den durchgeführten Operationen dieser Schichten (entweder strenge Faltung oder Kreuzkorrelation) unberührt.
* Wenn ein Element in einer Merkmalskarte ein grösseres rezeptives Feld benötigt, um breitere Merkmale in der Eingabe zu erkennen, kann ein tieferes Netz in Betracht gezogen werden.




## Übungen (optional)

1. Konstruieren Sie ein Bild `X` mit diagonalen Kanten.
    - A. Was passiert, wenn man den Kernel `K` aus diesem Abschnitt darauf anwendet?
    - B. Was geschieht, wenn man `X` transponiert?
    - C. Was passiert, wenn man `K` transponiert?
2. Welche Fehlermeldung erhalten Sie, wenn Sie versuchen, den Gradienten für die Klasse "Conv2D", die wir erstellt haben, automatisch zu finden?
3. Wie stellt man eine Kreuzkorrelationsoperation als Matrixmultiplikation dar, indem man den Eingabe- und den Kerneltensor ändert?
4. Entwerfen Sie einige Kernel manuell.
    - A. Wie sieht die Form eines Kernels für die zweite Ableitung aus?
    - B. Wie lautet der Kernel für ein Integral?
    - C. Wie groß muss ein Kernel mindestens sein, um eine Ableitung vom Grad $d$ zu erhalten?